In [ ]:
notebook_path = "Projects/QuantumFlow/notebooks"
try:
    import os
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/My Drive/" + notebook_path)
except:
    pass

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
%matplotlib inline

tf.enable_eager_execution()

import ipywidgets as widgets
from IPython.display import display

import sys
sys.path.append('../')

from quantumflow.generate_datasets import generate_potentials
from quantumflow.calculus_utils import integrate, laplace, np_integrate
from quantumflow.numerov_solver import *
from quantumflow.colab_train_utils import load_hyperparameters, test_colab_devices, InputPipeline

has_gpu, has_tpu = test_colab_devices()
if has_gpu: print("Found GPU")
if has_tpu: print("Found TPU")

!pip install -q ruamel.yaml
data_dir = "../data"

# Algorithm Analysis

In [ ]:
experiment = 'recreate'
analyse_name = "train"

preview = 5
analysis_dataset_size = 100
figsize = (20, 3)
dpi = None

params = load_hyperparameters(os.path.join(data_dir, experiment, "hyperparams.config"), run_name=analyse_name, globals=globals())

params['dataset_size'] = analysis_dataset_size
tf.set_random_seed(params['seed'])

potential, x, h = generate_potentials(return_x=True, return_h=True, **params)
params['h'] = h

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(x, np.transpose(potential)[:, :preview]) # only plot first potentials
plt.title("Randomly Generated Potentials")
plt.xlabel("x / bohr")
plt.ylabel("Energy / hartree")
plt.grid(which='major', axis='y', linestyle='--')
plt.show()

In [ ]:
split_energies = find_split_energies(potential, params).numpy()

plt.figure(figsize=figsize, dpi=dpi)
bins = np.linspace(min(split_energies.flatten()), max(split_energies.flatten()), 100)

for i in range(split_energies.shape[1]):
    color = i/(split_energies.shape[1] - 1)
    plt.hist(split_energies[:, i], bins, alpha=0.7, color=[0.8*color**2, 0.8*(1-color)**2, 1.6*color*(1-color)], label= str(i) + " roots")

plt.xlabel('split energy / hartree')
plt.ylabel('count')
plt.legend(loc='upper right')
plt.title("Energies giving the intervals where there is exactly one eigenvalue")
plt.show()

In [ ]:
energies, wavefunctions = solve_schroedinger(potential, params)

fig, axs = plt.subplots(1, preview, figsize=(figsize[0], figsize[1]*2), dpi=dpi)

for i, plot in enumerate(wavefunctions.numpy()[:preview]**2 + energies.numpy()[:preview, np.newaxis, :]):
    for n, plot_single in enumerate(plot.transpose()):
        axs[i].plot(x, potential[i], 'k')
        axs[i].plot(x, np.ones(x.shape)*energies[i, n], ':k')
        axs[i].plot(x, plot_single, 'C' + str(i%10))
        axs[i].set_ylim([np.min(potential.numpy()[:preview]), max(np.max(energies.numpy()[:preview]*1.1), 0.5)])
        if i == 0: 
            axs[i].set_ylabel('energies, potential / hartree')
            axs[i].set_xlabel("x / bohr")
        else:
            axs[i].get_yaxis().set_visible(False)
#fig.suptitle('Numerov Solution Energies and Densities')
plt.show()

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)
for i, plot in enumerate(wavefunctions[:preview]):
    plt.plot(x, plot, 'C' + str(i%10))
plt.xlim(x.numpy()[[0, -1]])
plt.title('Numerov Solutions')
plt.show()

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)
plt.plot(x, np.transpose(wavefunctions[:, :, 0])[:, :preview]) # only plot first potentials
plt.title("Wavefunction N = 1")
plt.xlabel("x / bohr")
plt.ylabel("Energy / hartree")
plt.grid(which='major', axis='y', linestyle='--')
plt.show()

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(x, np.transpose(wavefunctions[:, :, 0]**2)[:, :preview]) # only plot first potentials
plt.title("Density N = 1")
plt.xlabel("x / bohr")
plt.ylabel("Energy / hartree")
plt.grid(which='major', axis='y', linestyle='--')
plt.show()

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(x, np.transpose(tf.reduce_sum(wavefunctions**2, axis=2))[:, :preview]) # only plot first potentials
plt.title("Density N = {}".format(params['n_orbitals']))
plt.xlabel("x / bohr")
plt.ylabel("Energy / hartree")
plt.grid(which='major', axis='y', linestyle='--')
plt.show()

In [ ]:
laplace_wavefunctions = laplace(wavefunctions, params['h'])
kinetic_energy_densities = -0.5*wavefunctions*laplace_wavefunctions

kinetic_energies = integrate(kinetic_energy_densities, params['h'])

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)
plt.plot(x, np.transpose(kinetic_energy_densities[:, :, 0])[:, :preview]) # only plot first potentials
plt.title("Kinetic Energy Density N = 1")
plt.xlabel("x / bohr")
plt.ylabel("Energy / hartree")
plt.grid(which='major', axis='y', linestyle='--')
plt.show()

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(x, np.transpose(tf.reduce_sum(kinetic_energy_densities, axis=2))[:, :preview]) # only plot first potentials
plt.title("Kinetic Energy Density N = {}".format(params['n_orbitals']))
plt.xlabel("x / bohr")
plt.ylabel("Energy / hartree")
plt.grid(which='major', axis='y', linestyle='--')
plt.show()

In [ ]:
potential_energy_densities = tf.expand_dims(potential, axis=2)*wavefunctions**2
potential_energies = integrate(potential_energy_densities, params['h'])

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(x, np.transpose(potential_energy_densities[:, :, 0])[:, :preview]) # only plot first potentials
plt.title("Potential Energy Density N = 1")
plt.xlabel("x / bohr")
plt.ylabel("Energy / hartree")
plt.grid(which='major', axis='y', linestyle='--')
plt.show()

plt.figure(figsize=figsize, dpi=dpi)
plt.plot(x, np.transpose(tf.reduce_sum(potential_energy_densities, axis=2))[:, :preview]) # only plot first potentials
plt.title("Potential Energy Density N = {}".format(params['n_orbitals']))
plt.xlabel("x / bohr")
plt.ylabel("Energy / hartree")
plt.grid(which='major', axis='y', linestyle='--')
plt.show()


In [ ]:
kcalmol_per_hartree = 627.51
kinetic_energies_indirect = energies - potential_energies

print('N=1 energy:                 ', energies[:preview, 0].numpy())
print('N=1 potential energy:       ', potential_energies[:preview, 0].numpy())
print('N=1 kinetic energy:         ', kinetic_energies[:preview, 0].numpy())
print('N=1 kinetic energy indirect:', kinetic_energies_indirect[:preview, 0].numpy())

err = np.abs(kinetic_energies - kinetic_energies_indirect).flatten()*kcalmol_per_hartree
plt.figure(figsize=figsize, dpi=dpi)
plt.hist(err, bins=100)
plt.plot([np.mean(err), np.mean(err)], [0, np.sqrt(params['dataset_size']*params['n_orbitals'])], 'r')
plt.title("N = 1 Kinetic Energy Direct/Indirect Error")
plt.xlabel('error / kcal/mol')
plt.show()

In [ ]:
total_energy = tf.reduce_sum(energies, axis=-1)
print('total energy:                 ', total_energy[:preview].numpy())

potential_energy = tf.reduce_sum(potential_energies, axis=-1)
print('total potential energy:       ', potential_energy[:preview].numpy())

kinetic_energy = tf.reduce_sum(kinetic_energies, axis=-1)
print('total kinetic energy direct:  ', kinetic_energy[:preview].numpy())

kinetic_energy_indirect = tf.reduce_sum(kinetic_energies_indirect, axis=-1)
print('total kinetic energy indirect:', kinetic_energy_indirect[:preview].numpy())

err = np.abs(kinetic_energy - kinetic_energy_indirect).flatten()*kcalmol_per_hartree
plt.figure(figsize=figsize, dpi=dpi)
plt.hist(err, bins=100)
plt.plot([np.mean(err), np.mean(err)], [0, np.sqrt(params['dataset_size'])], 'r')
plt.title("Total Kinetic Energy Direct/Indirect Error")
plt.xlabel('error / kcal/mol')
plt.show()

# Dataset Analysis

In [ ]:
experiment = 'recreate'
analyse_name = 'train'
fine_name = 'fine'

N = 1
preview = 5
figsize = (20, 3)
dpi = None

params_train = load_hyperparameters(os.path.join(data_dir, experiment, "hyperparams.config"), run_name=analyse_name, globals=globals())
params_fine = load_hyperparameters(os.path.join(data_dir, experiment, "hyperparams.config"), run_name=fine_name, globals=globals())
dataset_train = InputPipeline(N, os.path.join(data_dir, experiment, params_train['filename']))
dataset_fine = InputPipeline(N, os.path.join(data_dir, experiment, params_fine['filename']))

In [ ]:
kcalmol_per_hartree = 627.51
err = np.abs(dataset_train.energies - dataset_fine.energies).flatten()*kcalmol_per_hartree
plt.figure(figsize=figsize, dpi=dpi)
plt.hist(err, bins=100)
plt.plot([np.mean(err), np.mean(err)], [0, np.sqrt(dataset_train.dataset_size)], 'r')
plt.title("Total Energy Discretisation Error")
plt.xlabel('error / kcal/mol')
plt.show()